In [1]:
!pip install neo4j transformers sentence-transformers

In [2]:
from neo4j import GraphDatabase
from sentence_transformers import SentenceTransformer
from transformers import pipeline


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
class Neo4jKnowledgeGraph:
    def _init_(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def create_node(self, label, properties):
        with self.driver.session() as session: #indented this line
            query = f"CREATE (n:{label} $props)"  # Corrected query
            session.run(query, {"props": properties})  # Pass properties as a parameter

    def create_relationship(self, node1, node2, relationship_type):
        with self.driver.session() as session:
            query = """
            MATCH (a {name: $node1}), (b {name: $node2})
            CREATE (a)-[r:%s]->(b)
            """ % relationship_type
            session.run(query, {'node1': node1, 'node2': node2})

    def query_graph(self, query, params=None):
        with self.driver.session() as session:
            return session.run(query, params)

In [4]:
# Step 1: Construct a knowledge graph from text documents

def construct_knowledge_graph(neo4j_kg, documents):
    for doc in documents:
        sentences = doc.split('.')
        for sentence in sentences:
            entities = extract_entities(sentence)
            if len(entities) >= 2:
                for i in range(len(entities) - 1):
                    # Use the run method to execute Cypher queries for node creation
                    neo4j_kg.run("CREATE (e:Entity {name: $name})", name=entities[i])
                    neo4j_kg.run("CREATE (e:Entity {name: $name})", name=entities[i + 1])
                    # Use the run method to execute Cypher queries for relationship creation
                    neo4j_kg.run("MATCH (a:Entity {name: $name1}), (b:Entity {name: $name2}) CREATE (a)-[:RELATED_TO]->(b)", name1=entities[i], name2=entities[i + 1])

# Helper to extract entities (dummy function, replace with NER)
def extract_entities(sentence):
    # Dummy split of words, replace with actual NER processing
    return sentence.split()[:2]  # This should return actual entities from sentence

In [5]:
# Step 2: Expand SEC Knowledge Graph
def expand_knowledge_graph(neo4j_kg, additional_data):
    for data in additional_data:
        entities = extract_entities(data)
        if len(entities) >= 2:
            # Use the run method to execute Cypher queries for node creation
            neo4j_kg.run("CREATE (e:Entity {name: $name})", name=entities[0])
            neo4j_kg.run("CREATE (e:Entity {name: $name})", name=entities[1])
            # Use the run method to execute Cypher queries for relationship creation
            neo4j_kg.run("MATCH (a:Entity {name: $name1}), (b:Entity {name: $name2}) CREATE (a)-[:EXPANDED_WITH]->(b)", name1=entities[0], name2=entities[1])

In [23]:
def query_knowledge_graph(neo4j_kg, entity):
    # Use backticks for the name property and case-insensitive search
    query = "MATCH (n {`name`: $entity})-[*]-(related) RETURN DISTINCT related.`name` as name"
    # Use the run method to execute the query with toLower() for case-insensitivity
    result = neo4j_kg.run(query, {"entity": entity.lower()})
    return [record["name"] for record in result]

In [24]:
# Step 4: Prepare text for RAG and chat with the graph
def prepare_text_for_rag(knowledge_data):
    model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
    embeddings = model.encode(knowledge_data)
    return embeddings

def chat_with_knowledge_graph(neo4j_kg, query_text):
    rag_pipeline = pipeline('question-answering', model='facebook/bart-large-cnn')

    # Convert query_text to lowercase for case-insensitive search
    related_entities = query_knowledge_graph(neo4j_kg, query_text.lower())

    # Check if related_entities is empty and provide a default response
    if not related_entities:
        return "I'm sorry, I don't have any information about that."

    knowledge_text = " ".join(related_entities)

    answer = rag_pipeline(question=query_text, context=knowledge_text)
    return answer["answer"]


In [25]:

from py2neo import Graph # Add this line to import the Graph class from the py2neo library

if __name__ == "__main__": # Fixed the typo here
    # Connect to Neo4j using the Graph object from py2neo
    neo4j_kg = Graph(uri="neo4j+s://eacc3e56.databases.neo4j.io", auth=("neo4j", "neo4j"))

    # Sample text data (replace with actual documents)
    documents = ["John is a professor at Stanford. He teaches AI.", "AI is a fascinating field of study."]

    # Step 1: Construct Knowledge Graph
    construct_knowledge_graph(neo4j_kg, documents)

    # Step 2: Expanding the SEC Knowledge Graph
    additional_data = ["John researches machine learning.", "Machine learning is a subset of AI."]
    expand_knowledge_graph(neo4j_kg, additional_data)

    # Step 3: Chat with the Knowledge Graph
    query_text = "Tell me about AI."
    response = chat_with_knowledge_graph(neo4j_kg, query_text)

    print(f"Chatbot Answer: {response}")

Some weights of BartForQuestionAnswering were not initialized from the model checkpoint at facebook/bart-large-cnn and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Chatbot Answer: I'm sorry, I don't have any information about that.
